In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

2024-06-23 22:38:23.209261: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-23 22:38:23.279860: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-23 22:38:23.280870: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 22:38:24.455544: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_dir='dataset/train'
test_dir='dataset/test'

In [3]:
def load_dataset(directory):
  image_paths=[]
  labels=[]

  for label in os.listdir(directory):
    for filename in os.listdir(directory+"/"+label):
      image_path=os.path.join(directory,label,filename)
      image_paths.append(image_path)
      labels.append(label)
    print(label,"compelted")

  return image_paths, labels

In [4]:
train=pd.DataFrame()
train['image'], train['label']=load_dataset(train_dir)

train=train.sample(frac=1).reset_index(drop=True)
train.head()

fear compelted
disgust compelted
neutral compelted
surprise compelted
sad compelted
happy compelted
angry compelted


,image,label
0,dataset/train/happy/Training_23853027.jpg,happy
1,dataset/train/angry/Training_20825412.jpg,angry
2,dataset/train/neutral/Training_37475756.jpg,neutral
3,dataset/train/sad/Training_3726599.jpg,sad
4,dataset/train/angry/Training_99547128.jpg,angry


In [5]:
test=pd.DataFrame()
test['image'], test['label']=load_dataset(test_dir)
test.head()

fear compelted
disgust compelted
neutral compelted
surprise compelted
sad compelted
happy compelted
angry compelted


,image,label
0,dataset/test/fear/PublicTest_80861502.jpg,fear
1,dataset/test/fear/PrivateTest_7183558.jpg,fear
2,dataset/test/fear/PublicTest_44296746.jpg,fear
3,dataset/test/fear/PublicTest_65218414.jpg,fear
4,dataset/test/fear/PublicTest_19459861.jpg,fear


In [6]:
def extract_features(images):
  features=[]
  for image in tqdm(images):
    img=load_img(image,grayscale=True)
    img=np.array(img)
    features.append(img)
  features=np.array(features)
  features=features.reshape(len(features),48,48,1)
  return features


In [7]:
train_features=extract_features(train['image'])

  0%|          | 0/28709 [00:00<?, ?it/s]

In [8]:
test_features=extract_features(test['image'])

  0%|          | 0/7178 [00:00<?, ?it/s]

In [9]:
x_train=train_features/255.0
x_test=test_features/255.0

In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(train['label'])
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [11]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test, num_classes=7)

In [12]:
y_train[1]

array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [13]:
input_shape=(48,48,1)
output_class=7


In [14]:
from keras.models import load_model
model1=load_model('Model1_2_sigmoid.h5')
model2=load_model('model2.h5')
model3=load_model('model3_1.h5')

In [15]:
predictions_model1=model1.predict(x_train)


2024-06-23 22:38:33.115638: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 264582144 exceeds 10% of free system memory.


898/898 [==============================] - 58s 65ms/step


In [16]:
predictions_model2=model2.predict(x_train)

2024-06-23 22:39:32.310818: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 264582144 exceeds 10% of free system memory.


898/898 [==============================] - 94s 105ms/step


In [18]:
predictions_model3=model3.predict(x_train)

2024-06-23 22:41:42.207877: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 264582144 exceeds 10% of free system memory.


898/898 [==============================] - 37s 41ms/step


In [19]:
type(predictions_model3)

numpy.ndarray

In [20]:
predictions_model1[0]

array([0.50373065, 0.01405159, 0.25461024, 0.9795197 , 0.90908384,
       0.6606964 , 0.0894388 ], dtype=float32)

In [21]:
predictions_model2[0]

array([0.29122275, 0.00328792, 0.13610412, 0.9971039 , 0.94872564,
       0.30795464, 0.20054792], dtype=float32)

In [22]:
predictions_model3[0]

array([0.93401617, 0.01131978, 0.54349905, 0.99995744, 0.8804907 ,
       0.05937964, 0.9779617 ], dtype=float32)

In [23]:
predictions=predictions_model1+predictions_model2+predictions_model3

In [24]:
predictions[0]

array([1.7289696 , 0.02865929, 0.9342134 , 2.976581  , 2.7383003 ,
       1.0280308 , 1.2679484 ], dtype=float32)

In [25]:
predictions=predictions/3

In [26]:
predictions[0]

array([0.5763232 , 0.0095531 , 0.31140447, 0.9921937 , 0.91276675,
       0.3426769 , 0.42264947], dtype=float32)

In [27]:
true_class=[]
for i in y_train:
    j=i.tolist()
    true_class.append(j.index(1))


In [28]:
true_class=np.array(true_class)

In [29]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy')
classifier.fit(predictions, true_class)

RandomForestClassifier(criterion='entropy')

In [31]:
y_features1 = model1.predict(x_test)

225/225 [==============================] - 15s 68ms/step


In [32]:
y_features2=model2.predict(x_test)

225/225 [==============================] - 22s 99ms/step


In [33]:
y_features3=model3.predict(x_test)

225/225 [==============================] - 9s 40ms/step


In [34]:
y_features=y_features1+y_features2+y_features3

In [35]:
y_features=y_features/3

In [36]:
y_features[0]

array([0.62956774, 0.01254743, 0.9374326 , 0.3385407 , 0.250517  ,
       0.50303686, 0.6246478 ], dtype=float32)

In [37]:
valid_class=[]
for i in y_test:
    j=i.tolist()
    valid_class.append(j.index(1))
test_answers=np.array(valid_class)

In [38]:
y_pred = classifier.predict(y_features)

In [39]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 555    8  112   45  103  119   16]
 [  35   62    3    1    3    6    1]
 [ 111    4  504   28  102  196   79]
 [  44    0   41 1510   92   57   30]
 [ 110    2   73   82  740  204   22]
 [ 147    0  170   70  213  627   20]
 [  21    1   87   43   20   15  644]]


0.6466982446363889

In [40]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(predictions, true_class)

SVC(kernel='linear', random_state=0)

In [41]:
y_pred = classifier.predict(y_features)

In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 555    8  112   45  103  119   16]
 [  35   62    3    1    3    6    1]
 [ 111    4  504   28  102  196   79]
 [  44    0   41 1510   92   57   30]
 [ 110    2   73   82  740  204   22]
 [ 147    0  170   70  213  627   20]
 [  21    1   87   43   20   15  644]]


0.6373641682920034

In [43]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(predictions,true_class)

SVC(random_state=0)

In [44]:
y_pred = classifier.predict(y_features)

In [45]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 555    8  112   45  103  119   16]
 [  35   62    3    1    3    6    1]
 [ 111    4  504   28  102  196   79]
 [  44    0   41 1510   92   57   30]
 [ 110    2   73   82  740  204   22]
 [ 147    0  170   70  213  627   20]
 [  21    1   87   43   20   15  644]]


0.6437726386179995

In [46]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(predictions,true_class)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [47]:
y_pred = classifier.predict(y_features)

In [48]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 555    8  112   45  103  119   16]
 [  35   62    3    1    3    6    1]
 [ 111    4  504   28  102  196   79]
 [  44    0   41 1510   92   57   30]
 [ 110    2   73   82  740  204   22]
 [ 147    0  170   70  213  627   20]
 [  21    1   87   43   20   15  644]]


0.6074115352465868

In [49]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 20, metric = 'minkowski', p = 2)
classifier.fit(predictions,true_class)

KNeighborsClassifier(n_neighbors=20)

In [50]:
y_pred = classifier.predict(y_features)

In [51]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 555    8  112   45  103  119   16]
 [  35   62    3    1    3    6    1]
 [ 111    4  504   28  102  196   79]
 [  44    0   41 1510   92   57   30]
 [ 110    2   73   82  740  204   22]
 [ 147    0  170   70  213  627   20]
 [  21    1   87   43   20   15  644]]


0.6391752577319587

In [52]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(predictions,true_class)

LogisticRegression(random_state=0)

In [53]:
y_pred = classifier.predict(y_features)

In [54]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 555    8  112   45  103  119   16]
 [  35   62    3    1    3    6    1]
 [ 111    4  504   28  102  196   79]
 [  44    0   41 1510   92   57   30]
 [ 110    2   73   82  740  204   22]
 [ 147    0  170   70  213  627   20]
 [  21    1   87   43   20   15  644]]


0.6369462245750905

In [55]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(predictions,true_class)

GaussianNB()

In [56]:
y_pred = classifier.predict(y_features)

In [57]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 555    8  112   45  103  119   16]
 [  35   62    3    1    3    6    1]
 [ 111    4  504   28  102  196   79]
 [  44    0   41 1510   92   57   30]
 [ 110    2   73   82  740  204   22]
 [ 147    0  170   70  213  627   20]
 [  21    1   87   43   20   15  644]]


0.6099191975480636